<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/CNN_model_big_dense_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 

     |████████████████████████████████| 1.9 MB 33.4 MB/s 
     |████████████████████████████████| 182 kB 45.4 MB/s 
     |████████████████████████████████| 166 kB 71.0 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 166 kB 66.5 MB/s 
     |████████████████████████████████| 162 kB 30.1 MB/s 
     |████████████████████████████████| 162 kB 78.9 MB/s 
     |████████████████████████████████| 158 kB 70.6 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 73.4 MB/s 
     |████████████████████████████████| 157 kB 61.8 MB/s 
     |████████████████████████████████| 157 kB 73.8 MB/s 
     |████████████████████████████████| 157 kB 74.4 MB/s 
     |████████████████████████████████| 157 kB 79.9 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 156 kB 78.8 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: h

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras import layers

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width)  )

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [7]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
class_names == test_class_names
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [32]:
num_classes = len(class_names)
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal_and_vertical"),tf.keras.layers.RandomRotation(0.2),tf.keras.layers.RandomZoom(0.1),])
model = Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  data_augmentation,
  layers.Conv2D(16, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(512, 3, activation='relu'),
  layers.MaxPooling2D(),
  #layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(1028, activation='relu'),
  layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])

In [33]:
model.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [34]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 sequential_6 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 111, 111, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                 

In [20]:
'''model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/Research Internship/augmentation/my_best_model.epoch{epoch:02d}-val_accuracy{val_accuracy:.2f}",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)'''

'model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(\n    filepath="/content/drive/MyDrive/Research Internship/augmentation/my_best_model.epoch{epoch:02d}-val_accuracy{val_accuracy:.2f}",\n    save_weights_only=False,\n    monitor=\'val_accuracy\',\n    mode=\'max\',\n    save_best_only=True)'

In [35]:
wandb.init(
      # Set the project where this run will be logged
      project="CNN_particle_classification", 
      name = "big dense layer",
)

In [36]:
%%time
epochs=200
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[WandbCallback()]
)

Epoch 1/200
22/22 [==============================] - ETA: 0s - loss: 1.4114 - accuracy: 0.3793

wandb: Adding directory to artifact (/content/wandb/run-20221104_072028-1cbfrzs6/files/model-best)... Done. 0.2s


22/22 [==============================] - 9s 311ms/step - loss: 1.4114 - accuracy: 0.3793 - val_loss: 1.1843 - val_accuracy: 0.3886
Epoch 2/200
22/22 [==============================] - 3s 130ms/step - loss: 1.1141 - accuracy: 0.5469 - val_loss: 1.2049 - val_accuracy: 0.5371
Epoch 3/200
22/22 [==============================] - ETA: 0s - loss: 1.1002 - accuracy: 0.5866

wandb: Adding directory to artifact (/content/wandb/run-20221104_072028-1cbfrzs6/files/model-best)... Done. 0.2s


22/22 [==============================] - 7s 280ms/step - loss: 1.1002 - accuracy: 0.5866 - val_loss: 1.1414 - val_accuracy: 0.5257
Epoch 4/200
22/22 [==============================] - ETA: 0s - loss: 1.0182 - accuracy: 0.6165

...

22/22 [==============================] - 7s 293ms/step - loss: 0.1263 - accuracy: 0.9588 - val_loss: 0.1686 - val_accuracy: 0.9543
Epoch 194/200
22/22 [==============================] - 3s 121ms/step - loss: 0.1230 - accuracy: 0.9602 - val_loss: 0.3721 - val_accuracy: 0.8857
Epoch 195/200
22/22 [==============================] - 3s 122ms/step - loss: 0.1080 - accuracy: 0.9645 - val_loss: 0.3250 - val_accuracy: 0.9086
Epoch 196/200
22/22 [==============================] - 3s 122ms/step - loss: 0.0958 - accuracy: 0.9631 - val_loss: 0.2890 - val_accuracy: 0.9086
Epoch 197/200
22/22 [==============================] - ETA: 0s - loss: 0.0947 - accuracy: 0.9673

wandb: Adding directory to artifact (/content/wandb/run-20221104_072028-1cbfrzs6/files/model-best)... Done. 0.2s


22/22 [==============================] - 6s 273ms/step - loss: 0.0947 - accuracy: 0.9673 - val_loss: 0.1612 - val_accuracy: 0.9371
Epoch 198/200
22/22 [==============================] - ETA: 0s - loss: 0.0811 - accuracy: 0.9702

wandb: Adding directory to artifact (/content/wandb/run-20221104_072028-1cbfrzs6/files/model-best)... Done. 0.2s


22/22 [==============================] - 7s 299ms/step - loss: 0.0811 - accuracy: 0.9702 - val_loss: 0.1107 - val_accuracy: 0.9714
Epoch 199/200
22/22 [==============================] - 3s 121ms/step - loss: 0.0636 - accuracy: 0.9759 - val_loss: 0.3069 - val_accuracy: 0.8743
Epoch 200/200
22/22 [==============================] - 3s 122ms/step - loss: 0.0676 - accuracy: 0.9744 - val_loss: 0.3422 - val_accuracy: 0.8914
CPU times: user 11min 53s, sys: 1min 42s, total: 13min 36s
Wall time: 15min 57s


In [37]:
wandb.finish()

accuracy,▁▂▃▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████▇███████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▃▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁▂▂▁
val_accuracy,▁▁▂▃▄▅▆▅▅▆▆▇▆▆▅▆▇█▇▇▆▇▇▇▇█▇█▇█▇█▇█▇▇▇█▇▇
val_loss,█▇▆▆▅▄▃▄▄▅▂▃▂▂▄▂▂▁▂▂▅▂▂▂▁▂▃▄▂▁▂▁▂▁▂▆▄▂▂▁
accuracy,0.97443
best_epoch,197
best_val_loss,0.11073
epoch,199
loss,0.06762
val_accuracy,0.89143


In [31]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
